# Source-to-source translation
**Generating adjoint code for Numpy expressions with 'ad_transformer'**.

In [ ]:
import ast
import inspect
import numpy
from ad_transformer import transform
from draw_ast import draw_AST

## 1st Example: A*B+C

In [ ]:
# First test function
def simple_expr(A: numpy.ndarray, B: numpy.ndarray, C: numpy.ndarray) -> numpy.ndarray:
    return A @ B + C

In [ ]:
# visualize abstract syntax tree using ast.parse and ast.NodeVisitor
draw_AST(simple_expr)

In [ ]:
# Transform the function code
simple_expr_transformed = transform(simple_expr)
print(simple_expr_transformed)

### Explanation:
- The function name is extended with "_ad" and adjoints (context) of function arguments are included ("A_a", "B_a", ...).
- The original function is completely transformed to single-assignment code.
- No overwriting/incremental assignments of variables is considered for now. Hence no "tbr_stack" etc.
- Only functions that return a single (ndarray) variable are considered for now.
- Adjoints of all intermediate v_i are zero-initialized.
- The adjoint of the return value ("v4_a") is seeded to ones.
- For derivatives, all v_i are treated as scalars (we are not differentiating matrices by matrices because that would result in tensors)
- All arithmetic is element-wise. Only the @ operator maps to matrix products.
- The primal result is returned together with all adjoints in a tuple.

### Test:

In [ ]:
# Compile and execute it to make it visible
exec(compile(simple_expr_transformed, filename="<ast>", mode="exec"))

# active arguments
dims = (2,2)
A = 1.5 * numpy.ones(dims)
B = 2.0 * numpy.ones(dims)
C = 3.0 * numpy.ones(dims)
# initialize adjoints
A_a = numpy.zeros(dims)
B_a = numpy.zeros(dims)
C_a = numpy.zeros(dims)

# A*B+C
result, dfdA, dfdB, dfdC = simple_expr_ad(A, B, C, A_a, B_a, C_a)
print("primal result:\n {r}\n df/dA:\n {a}\n df/dB:\n {b}\n df/dC:\n {c}".format(r=result, a=dfdA, b=dfdB, c=dfdC))

## 2nd Example: Element-wise Sigmoid Function 
### sigmoid(X) = 1 / (1 + exp(-X))

In [ ]:
def sigmoid(A: numpy.ndarray) -> numpy.ndarray:
    denominator = numpy.ones(A.shape) + numpy.exp(-A)
    return numpy.divide(numpy.ones(A.shape), denominator)

In [ ]:
draw_AST(sigmoid)

'transform' already supports derivatives of some numpy functions and more can be added easily. 

In [ ]:
sigmoid_transformed = transform(sigmoid)
print(sigmoid_transformed)

## Test:

In [ ]:
dims = (3,3)
X = numpy.zeros(dims)
X_a = numpy.zeros(dims)
exec(compile(sigmoid_transformed, filename="<ast>", mode="exec"))

s_X, s_dX = sigmoid_ad(X, X_a)
# Note that d/dx sigmoid(x) = (1-sigmoid(x))*sigmoid(x)
print("primal result:\n {s}\n df/dX:\n {ds}".format(s=s_X, ds=s_dX))


In [ ]:
# Plot sigmoid and its derivative 
import matplotlib.pyplot as plt
num = 50
x_vec = numpy.linspace(-10, 10, num=num)
x_plot = numpy.zeros(num)
x_a_vec = numpy.zeros(num)
x_a_plot = numpy.zeros(num)

x_plot, x_a_plot = sigmoid_ad(x_vec, x_a_vec)
plt.plot(x_vec, x_plot, label="sigmoid(x)")
plt.plot(x_vec, x_a_plot, "--", label="d/dx sigmoid(x)")
plt.legend();

TODO:
- make adjoint direction input (entry-wise differentiation)
- add rules for transpose and inverse
- add validation with finite difference
- add GLS example


- Example for Ax+b, xAx+bx, (A^TA)^-1 A^Tx